In [1]:
import wandb
import pandas as pd
import datetime
import tqdm

username = "memmelma"
project = "vit-vo"

filter_metric = "Objective/eval_all"
filter_operation = "min"

metrics = ["Objective/eval_forward", "Objective/eval_left", "Objective/eval_right", "eval_all_unified_depth/aux_depth_loss",
           "Objective/train"]

configs = ["config.CHECKPOINT_FOLDER", "config.VO.MODEL.visual_backbone", "config.VO.MODEL.pretrain_backbone",
           "config.VO.MODEL.cls_action", "config.VO.TRAIN.depth_aux_loss"]

step_threshold = 100

In [2]:
# retreive runs

run_ids = []
run_names = []
run_ckpts = []

api = wandb.Api()
runs = api.runs(f"{username}/{project}")

runs_df = pd.DataFrame(columns=["name", "id", "state", *configs, filter_metric, *metrics])

for run in tqdm.tqdm(runs):
    
    run_dict = {}
    
    # add base attributes
    run_dict["name"] = run.name.split("[")[0]
    run_dict["id"] = run.id 
    run_dict["state"] = run._state
    
    # add config
    for config in configs:
        # traverse config keys
        run_config_tmp = run.config
        for config_iter in config.split('.'):
            run_config_tmp = run_config_tmp[config_iter]
        run_dict[config] = run_config_tmp
    
    # add filter_operation filter_metric
    run_history = run.history()
    select = run_history["_step"]<step_threshold
    
    if filter_operation == "min":
        filter_metric_idx = run_history[filter_metric][select].argmin()
    elif filter_operation == "max":
        filter_metric_idx = run_history[filter_metric][select].argmax()
    else:
        break
        
    run_dict[filter_metric] = run_history[filter_metric][filter_metric_idx]
    
    # add other metrics corresponding to filter_operation filter_metric
    for metric in metrics:
        run_dict[metric] = run_history[metric][filter_metric_idx]
    
    runs_df = runs_df.append(run_dict, ignore_index=True)
    
runs_df

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:11<00:00,  2.16it/s]


,name,id,state,config.CHECKPOINT_FOLDER,config.VO.MODEL.visual_backbone,config.VO.MODEL.pretrain_backbone,config.VO.MODEL.cls_action,config.VO.TRAIN.depth_aux_loss,Objective/eval_all,Objective/eval_forward,Objective/eval_left,Objective/eval_right,eval_all_unified_depth/aux_depth_loss,Objective/train
0,vo_vit_s_blind_act,2t4idcqk,running,train_log/vit/unique/vo_vit_s_blind_act/checkp...,small,None,True,0,0.016434,0.015654,0.005300,0.005377,0,0.015022
1,vo_vit_s_blind,19pui3rf,running,train_log/vit/unique/vo_vit_s_blind/checkpoints,small,None,False,0,0.472582,0.058887,0.299951,0.306638,0,0.458435
2,vo_vit_s_in21k_rgb,1ybxrwyi,crashed,train_log/vit/unique/vo_vit_s_in21k_rgb/checkp...,small,in21k,False,0,0.004580,0.004169,0.001484,0.001581,0,0.002086
3,vo_vit_s_scratch_act_rgbd,1nfvika9,running,train_log/vit/unique/vo_vit_s_scratch_act_rgbd...,small,None,True,0,0.002897,0.001736,0.001118,0.001218,0,0.000496
4,vo_vit_s_scratch_rgb,3ggek0u8,running,train_log/vit/unique/vo_vit_s_scratch_rgb/chec...,small,None,False,0,0.006734,0.007211,0.001613,0.001703,0,0.005664
5,vo_vit_s_scratch_d,1k445jno,finished,train_log/vit/unique/vo_vit_s_scratch_d/checkp...,small,None,False,0,0.003884,0.001654,0.001339,0.001628,0,0.002703
6,vo_vit_s_in21k_rgb_freeze,3f4i9631,finished,train_log/vit/unique/vo_vit_s_in21k_rgb_freeze...,small,in21k,False,0,0.464150,0.033142,0.300392,0.310057,0,0.447203
7,vo_vit_s_scratch_act_rgb,3ow4ka5n,finished,train_log/vit/unique/vo_vit_s_scratch_act_rgb/...,small,None,True,0,0.016549,0.015804,0.005313,0.005395,0,0.017772
8,vo_vit_s_in21k_act_rgb_freeze,3vrhxq4u,finished,train_log/vit/unique/vo_vit_s_in21k_act_rgb_fr...,small,in21k,True,0,0.019548,0.016824,0.006148,0.005472,0,0.015847
9,vo_vit_s_in21k_rgb_auxd,ttyf4ilc,finished,train_log/vit/unique/vo_vit_s_in21k_rgb_auxd/c...,small,in21k,True,1,0.030583,0.027764,0.016139,0.016360,0.008755,0.028904


In [16]:
dt = datetime.datetime.now().strftime("%Y%m%d_%H%M%S%f")
filename = f"experiments_2020_{dt}"

In [15]:
# # save df as csv file
# extension = ".csv"
# runs_df.to_csv(filename+extension)

In [ ]:
!apt-get update -y
!apt-get install -y python-xlsxwriter
!pip install openpyxl

In [13]:
# save df as excel file
extension = ".xlsx"
excel_writer = pd.ExcelWriter(filename+extension)
runs_df.to_excel(excel_writer, sheet_name='Sheet1')